In [4]:
from datasets import Dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [5]:
train_dataset=pd.read_json("dataset.json")

In [6]:
test_dataset=pd.read_json("test.json")

In [7]:
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [8]:
model_name = "google/mt5-small"
model =AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pyth

In [10]:
def preprocess(data):
  input=tokenizer(data['incorrect'],padding='max_length',truncation=True,max_length=128)
  labels = tokenizer(data['correct'], padding='max_length', truncation=True,max_length=128)
  input["labels"] = labels["input_ids"]
  return input


encoded_train_dataset = train_dataset.map(preprocess, batched=True)
encoded_test_dataset = test_dataset.map(preprocess, batched=True)

print(encoded_train_dataset[0])
print(encoded_test_dataset[0])

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

{'incorrect': 'مدون', 'correct': 'مدیون', 'input_ids': [548, 34126, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [6289, 51178, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ2SEQ_LM"
)

lora_model = get_peft_model(model, lora_config)

In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/temp/result',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/logs',
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset
)

trainer.train()


Step,Training Loss
10,64.036700
20,64.014800
30,63.710700


TrainOutput(global_step=30, training_loss=63.920772298177084, metrics={'train_runtime': 716.4469, 'train_samples_per_second': 0.649, 'train_steps_per_second': 0.042, 'total_flos': 61590023700480.0, 'train_loss': 63.920772298177084, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/temp/model")
tokenizer.save_pretrained("/content/drive/MyDrive/temp/tokenizer")

# ذخیره مدل LoRA
lora_model.save_pretrained("/content/drive/MyDrive/temp/model")

In [14]:
from google.colab import files
import shutil

shutil.make_archive('/content/model', 'zip', '/content/drive/MyDrive/temp/model')
shutil.make_archive('/content/tokenizer', 'zip', '/content/drive/MyDrive/temp/tokenizer')
files.download('/content/model.zip')
files.download('/content/tokenizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>